# LANL Earhquake prediction

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd

pd.options.display.float_format = '{:,.10f}'.format

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

# Visualizations
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
sns.set()

In [117]:
FEATURES_BASE = '..data/processed/train/features_fast.csv'
FEATURES_BASE_DENOISE = 'features_den_fast.csv'
FEATURES_FOLDS_DENOISE = ''
FEATURES_ = 'features_slow.csv'

In [118]:
data_f = pd.read_csv(CACHE_DIR+FEATURES_F,index_col='id')
data_d_f = pd.read_csv(CACHE_DIR+FEATURES_D_F,index_col='id')
data_s = pd.read_csv(CACHE_DIR+FEATURES_S,index_col='id')

In [119]:
data_s.head(2)

,x__abs_energy,x__absolute_sum_of_changes,"x__agg_autocorrelation__f_agg_""mean""__maxlag_40","x__agg_autocorrelation__f_agg_""median""__maxlag_40","x__agg_autocorrelation__f_agg_""var""__maxlag_40","x__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","x__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","x__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","x__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","x__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""",...,x__symmetry_looking__r_0.9500000000000001,x__time_reversal_asymmetry_statistic__lag_1,x__time_reversal_asymmetry_statistic__lag_2,x__time_reversal_asymmetry_statistic__lag_3,x__value_count__value_-1,x__value_count__value_0,x__value_count__value_1,x__variance,x__variance_larger_than_standard_deviation,y
id,,,,,,,,,,,,,,,,,,,,,
0,"10,247.0000000000","391,980.0000000000",-0.0196169172,-0.0292105689,0.0771727933,11.4728051641,-0.1604799308,-0.0001705898,0.0000085674,15.5494230812,...,1.0000000000,0.2592567901,3.8303954772,-4.1183647346,"3,622.0000000000","5,741.0000000000","8,406.0000000000",26.0211102805,1.0000000000,1.4307971859
1,"31,377.0000000000","405,226.0000000000",-0.0296423639,-0.0473553483,0.0950035692,10.8740597116,-0.0420251683,-0.0000653812,0.0000126924,14.9988208375,...,1.0000000000,-6.4017320231,-0.2259793594,2.8093990426,"3,888.0000000000","5,853.0000000000","8,557.0000000000",43.4123094122,1.0000000000,1.3914988931


In [120]:
y_all = data_d_f['y']
X_all = data_d_f.drop('y',axis=1).join(data_s.drop('y',axis=1))
# X_all = data_s.drop('y',axis=1)
# X_all = data_d_f.drop('y',axis=1).join(data_s.drop('y',axis=1),rsuffix='s_').join(data_f.drop('y',axis=1),rsuffix='f_')
# X_all = data_f.drop('y',axis=1).join(data_s.drop('y',axis=1),rsuffix='s_')

In [121]:
print('data_f.shape:',data_f.shape)
print('data_s.shape:',data_s.shape)
print('X.shape:',X_all.shape)

data_f.shape: (4194, 116)
data_s.shape: (4194, 789)
X.shape: (4194, 903)


## Cleaning from NaN,infinity or too large values

In [122]:
if np.any(np.isnan(X_all)):
    X_all.fillna(0,inplace=True)

## Outer Cross-Validation split

In [123]:
from sklearn.model_selection import train_test_split
from src.settings import RANDOM_STATE
X, X_cross, y, y_cross = train_test_split(X_all, y_all, test_size=0.15, random_state=RANDOM_STATE)

## Feature selection

In [124]:
from src.utils.grid_search import GridSearch as gs
from src.grid import ARD_REGRESSION,BAYESIAN_RIDGE,ELASTIC_NET,ELASTIC_NET_CV,\
HUBER_REGRESSOR,KERNEL_RIDGE,LARS,LARS_CV,LASSO,LASSO_CV,LASSO_LARS,LASSO_LARS_CV,\
LASSO_LARS_IC,LINEAR_REGRESSION,MULTITASK_ELASTIC_NET,MULTITASK_ELASTIC_NET_CV,\
MULTITASK_LASSO,MULTITASK_LASSO_CV,ORTOGONAL_MATCHING_PURSUIT,ORTOGONAL_MATCHING_PURSUIT_CV,\
PASSIVE_AGGRESSIVE_REGRESSOR,RANSAC_REGRESSOR,RIDGE,RIDGE_CV,SGD_REGRESSOR,THEIL_SEN_REGRESSOR

In [125]:
from sklearn.linear_model import ARDRegression,BayesianRidge,ElasticNet,ElasticNetCV
from sklearn.linear_model import HuberRegressor, Lars,LarsCV,Lasso,LassoCV,LassoLars,LassoLarsCV
from sklearn.linear_model import LassoLarsIC,LinearRegression,MultiTaskElasticNet,MultiTaskElasticNetCV
from sklearn.linear_model import MultiTaskLasso, MultiTaskLassoCV, PassiveAggressiveRegressor, RANSACRegressor
from sklearn.linear_model import Ridge, RidgeCV, SGDRegressor, TheilSenRegressor
from sklearn.kernel_ridge import KernelRidge

In [134]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=3)

In [135]:
cv_dict = {}

In [138]:
%%time
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA

i=0
k_features = [50,100,200]
scalers = []

cv_dict['ridge'] = {}
for k_feature in k_features:
    cv_dict['ridge'][k_feature] = {
        'best_params':[],
        'best_score':[],
        'time_left':[],
        'mae':[]
    }
    for train_index, test_index in kf.split(X):
        print(k_feature)
        i+=1
    #     print('kfold.iteration=',i)
        X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]

    #     print('VarianceThreshold')
        var_selector = VarianceThreshold().fit(X_train)

    #     print('before.X.shape:',X_train.shape)
        X_train = X_train[X_train.columns[var_selector.get_support(indices=True)]]
    #     print('after.X.shape:',X_train.shape)

    #     print('SelectKBest')
#         pca = PCA(n_components=k_feature)
#         pca = pca.fit(X_train)
        
#         X_train = pd.DataFrame(data=pca.transform(X_train))
        
        
        kbest_selector = SelectKBest(f_regression, k=k_feature)
        kbest_selector = kbest_selector.fit(X_train, y_train)
        X_train = X_train[X_train.columns[kbest_selector.get_support(indices=True)]]

    #     print('Scaler')
#         scaler = StandardScaler()
        scaler = RobustScaler()
        scaler = scaler.fit(X_train)

        X_train_scale = scaler.transform(X_train)

        best_params, best_score, time_left = gs.best_parameters(Ridge(),RIDGE,X_train_scale,y_train)
        print(best_params,'loss:',best_score,'minutes_left:',time_left,'k_features_count:',k_feature)

    #     test dataset
        
        X_test = X_test[X_test.columns[var_selector.get_support(indices=True)]]
#         X_test = pca.transform(X_test)
        X_test = X_test[X_test.columns[kbest_selector.get_support(indices=True)]]
        X_test_scale = scaler.transform(X_test)

        model = Ridge(**best_params)
        model.fit(X_train_scale,y_train)
        prediction = model.predict(X_test_scale)

        cv_dict['ridge'][k_feature]['best_params'].append(best_params)
        cv_dict['ridge'][k_feature]['best_score'].append(best_score)
        cv_dict['ridge'][k_feature]['time_left'].append(time_left)
        cv_dict['ridge'][k_feature]['mae'].append(mean_absolute_error(y_test, prediction))

50
{'alpha': 1.5975975975975976, 'random_state': 42} loss: 2.0603147785511005 minutes_left: 0.3073072791099548 k_features_count: 50
50
{'alpha': 2.195195195195195, 'random_state': 42} loss: 2.087616549962874 minutes_left: 0.29331560532251993 k_features_count: 50
50
{'alpha': 10.362362362362363, 'random_state': 42} loss: 2.0872795436437803 minutes_left: 0.2965218226114909 k_features_count: 50
100
{'alpha': 15.342342342342343, 'random_state': 42} loss: 2.077441934417125 minutes_left: 0.4628608028093974 k_features_count: 100
100
{'alpha': 3.9879879879879883, 'random_state': 42} loss: 2.088354734451947 minutes_left: 0.46168492635091146 k_features_count: 100
100
{'alpha': 8.76876876876877, 'random_state': 42} loss: 2.087580281780778 minutes_left: 0.4685291330019633 k_features_count: 100
200
{'alpha': 59.36536536536537, 'random_state': 42} loss: 2.088282129101417 minutes_left: 1.2155125578244528 k_features_count: 200
200
{'alpha': 41.83583583583584, 'random_state': 42} loss: 2.1065007576427 

In [130]:
def print_cv(cv_dict):
    for model_name,cv in cv_dict.items():
        for key,val in cv.items():
            print(model_name,key,np.mean(val['mae']))

In [137]:
print_cv(cv_dict)

ridge 50 2.0737284539247804
ridge 100 2.0795526133288846
ridge 200 2.1005824361360768


In [139]:
print_cv(cv_dict)

ridge 50 2.0737284539247804
ridge 100 2.0795526133288846
ridge 200 2.1005824361360768


## Trees

In [106]:
num_folds = 5

def make_predictions(estimator, features, target, test=None, plot=True, lgb=False):
    """Train the estimator and make predictions for oof and test data."""
    folds = KFold(num_folds, shuffle=True, random_state=2019)
    oof_predictions = np.zeros(features.shape[0])
    if test is not None:
        sub_predictions = np.zeros(test.shape[0])
    for (train_index, valid_index) in folds.split(features, target):
        
        if lgb:
            estimator.fit(features[train_index], target[train_index],
                          early_stopping_rounds=100, verbose=False,
                          eval_set=[(features[train_index], target[train_index]),
                                    (features[valid_index], target[valid_index])])
        else:
            estimator.fit(features[train_index], target[train_index])
        oof_predictions[valid_index] = estimator.predict(features[valid_index]).flatten()
        if test is not None:
            sub_predictions += estimator.predict(test).flatten() / num_folds
    
    # Plot out-of-fold predictions vs actual values
    if plot:
        fig, axis = plt.subplots(1, 2, figsize=(12,5))
        ax1, ax2 = axis
        ax1.set_xlabel('actual')
        ax1.set_ylabel('predicted')
        ax2.set_xlabel('train index')
        ax2.set_ylabel('time to failure')
        ax1.scatter(target, oof_predictions, color='brown')
        ax1.plot([(0, 0), (20, 20)], [(0, 0), (20, 20)], color='blue')
        ax2.plot(target, color='blue', label='y_train')
        ax2.plot(oof_predictions, color='orange')
    if test is not None:
        return oof_predictions, sub_predictions
    else:
        return oof_predictions

In [113]:
fixed_params = {
    'objective': 'regression_l1',
    'boosting': 'gbdt',
    'verbosity': -1,
    'random_seed': 19,
    'n_estimators': 20000,
}

param_grid = {
    'learning_rate': [0.1, 0.05, 0.01, 0.005],
    'num_leaves': list(range(8, 92, 4)),
    'max_depth': [3, 4, 5, 6, 8, 12, 16, -1],
    'feature_fraction': [0.8, 0.85, 0.9, 0.95, 1],
    'subsample': [0.8, 0.85, 0.9, 0.95, 1],
    'lambda_l1': [0, 0.1, 0.2, 0.4, 0.6, 0.9],
    'lambda_l2': [0, 0.1, 0.2, 0.4, 0.6, 0.9],
    'min_data_in_leaf': [10, 20, 40, 60, 100],
    'min_gain_to_split': [0, 0.001, 0.01, 0.1],
}

In [ ]:
var_selector = VarianceThreshold().fit(X)

#     print('before.X.shape:',X_train.shape)
X_train = X_train[X_train.columns[var_selector.get_support(indices=True)]]
#     print('after.X.shape:',X_train.shape)

#     print('SelectKBest')
#         pca = PCA(n_components=k_feature)
#         pca = pca.fit(X_train)

#         X_train = pd.DataFrame(data=pca.transform(X_train))


kbest_selector = SelectKBest(f_regression, k=k_feature)
kbest_selector = kbest_selector.fit(X_train, y_train)
X_train = X_train[X_train.columns[kbest_selector.get_support(indices=True)]]

#     print('Scaler')
#         scaler = StandardScaler()
scaler = RobustScaler()
scaler = scaler.fit(X_train)

X_train_scale = scaler.transform(X_train)

In [115]:
import random
import lightgbm as lgb
dataset = lgb.Dataset(X, label=y)  # no need to scale features

best_score = 999
best_params= None
best_nrounds = None

for i in range(500):
    params = {k: random.choice(v) for k, v in param_grid.items()}
    params.update(fixed_params)
    result = lgb.cv(params, dataset, nfold=5, early_stopping_rounds=100,
                    stratified=False)
    
    if result['l1-mean'][-1] < best_score:
        best_score = result['l1-mean'][-1]
        best_params = params
        best_nrounds = len(result['l1-mean'])
        
    print(i,best_score,best_nrounds)

0 2.0586316906118114 1339
1 2.0559471740797006 53
2 2.0559471740797006 53
3 2.0559471740797006 53
4 2.0539913052066865 675
5 2.0539913052066865 675
6 2.0539913052066865 675


KeyboardInterrupt: 

In [ ]:
print("Best mean score: {:.4f}, num rounds: {}".format(best_score, best_nrounds))
print(best_params)
gb_oof = make_predictions(lgb.LGBMRegressor(**best_params), X.values, y, lgb=True)